#   Struct

###  Andrea Iskander Belkhir id : 511089

###  Beyza Özdemir id : 576145

In [1]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays

# R(arg)

it generate (and return)a rotation matrix,this funcion permit a 2D rotation retuning a 3 Dimension matrix or 3D rotation returning 4Dimension matrix... this part of the code can be separated in at least 2 more funcion (for a better reading at least)1 for 2D rot and the other for 3D rot.
the arg can be a singol parameter (an angol in radiant)or a 3 vector where we can use the norm(=sqr(dot_prod(arg,arg))) as the angle for the rotation and for the direction of the rotation the normalized value

In [2]:
function r(args...)
    args = collect(args)
    n = length(args)
    if n == 1 # rotation in 2D
        angle = args[1]; COS = cos(angle); SIN = sin(angle)
        mat = Matrix{Float64}(LinearAlgebra.I, 3, 3)
        mat[1,1] = COS;    mat[1,2] = -SIN;
        mat[2,1] = SIN;    mat[2,2] = COS;
    end

     if n == 3 # rotation in 3D
        mat = Matrix{Float64}(LinearAlgebra.I, 4, 4)
        angle = norm(args);
        if norm(args) != 0.0
			axis = args #normalize(args)
			COS = cos(angle); SIN= sin(angle)
			if axis[2]==axis[3]==0.0    # rotation about x
				mat[2,2] = COS;    mat[2,3] = -SIN;
				mat[3,2] = SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[3]==0.0   # rotation about y
				mat[1,1] = COS;    mat[1,3] = SIN;
				mat[3,1] = -SIN;    mat[3,3] = COS;
			elseif axis[1]==axis[2]==0.0    # rotation about z
				mat[1,1] = COS;    mat[1,2] = -SIN;
				mat[2,1] = SIN;    mat[2,2] = COS;
			else
				I = Matrix{Float64}(LinearAlgebra.I, 3, 3); u = axis
				Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
				UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
					 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
					 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
				mat[1:3,1:3]=COS*I+SIN*Ux+(1.0-COS)*UU
			end
		end
	end
	return mat
end

r (generic function with 1 method)

In [3]:
@btime r(0)

  91.444 ns (2 allocations: 256 bytes)


3×3 Matrix{Float64}:
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [4]:
@btime r(1,1,1)

  1.070 μs (12 allocations: 1.59 KiB)


4×4 Matrix{Float64}:
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [5]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.070 μs … 249.370 μs  ┊ GC (min … max): 0.00% … 98.38%
 Time  (median):     1.240 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.576 μs ±   3.850 μs  ┊ GC (mean ± σ):  4.08% ±  1.70%

  ▃▄▄▆██▅▂             ▁▁▁▂▁▁▂▃▃▃▃▂▂▃▂▂▂▂▂▂▁▁                 ▂
  █████████▆▇▇███▇▇▇▇▇▇████████████████████████████▇▇▇▆▆▄▄▅▆▅ █
  1.07 μs      Histogram: log(frequency) by time      2.95 μs <

 Memory estimate: 1.59 KiB, allocs estimate: 12.

# Improvment & changes

In [6]:
 function r(args...)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3D(args)
	end
	return mat
end

r (generic function with 1 method)

the new funcion r2D generate the rotation matrix for 2 dimension, the matrix in output has a dimension of 3 and the input (args) is a singol parameter

In [7]:
 @inline function r2D(args)
	angle = args[1]; COS = cos(angle); SIN = sin(angle)
	mat = MMatrix{3,3,Float64}(1I)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

r2D (generic function with 1 method)

the new funcion r3D generate the rotation matrix for 3 dimension, the matrix in output has a dimension of 4 and the input (args) is an array containing 3 elements, using those 3 element we can find the norm and using as rotation angle

In [8]:
@inline function r3D(args)
	mat =MMatrix{4,4,Float64}(1I)
	angle = norm(args);
	if angle != 0.0
		 axis = args #normalize(args)
		 COS = cos(angle); SIN= sin(angle)
		 if axis[2]==axis[3]==0.0    # rotation about x
			 mat = rX(mat,COS,SIN)
		 elseif axis[1]==axis[3]==0.0   # rotation about y
			 mat = rY(mat,COS,SIN)
		 elseif axis[1]==axis[2]==0.0    # rotation about z
			 mat = rZ(mat,COS,SIN)
		 else
			 mat = rAxis(mat,axis,COS,SIN)
		 end
	 end
	 return mat
end

r3D (generic function with 1 method)

In [9]:
@inline function rX(mat,COS,SIN)
	mat[2,2] = COS;    mat[2,3] = -SIN;
	mat[3,2] = SIN;    mat[3,3] = COS;
	return mat
end

rX (generic function with 1 method)

In [10]:
@inline function rY(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,3] = SIN;
	mat[3,1] = -SIN;    mat[3,3] = COS;
	return mat
end

rY (generic function with 1 method)

In [11]:
@inline function rZ(mat,COS,SIN)
	mat[1,1] = COS;    mat[1,2] = -SIN;
	mat[2,1] = SIN;    mat[2,2] = COS;
	return mat
end

rZ (generic function with 1 method)

"The speed of small SVectors, SMatrixs and SArrays is often > 10 × faster than Base.Array" quote from staticarray.jl ReadMe

In [12]:
@inline @fastmath function rAxis(mat,axis,COS,SIN)
	id = SMatrix{3,3,Float64}(1I); 
    u = axis
	Ux=[0 -u[3] u[2] ; u[3] 0 -u[1] ;  -u[2] u[1] 1]
	UU =[u[1]*u[1]    u[1]*u[2]   u[1]*u[3];
		 u[2]*u[1]    u[2]*u[2]   u[2]*u[3];
		 u[3]*u[1]    u[3]*u[2]   u[3]*u[3]]
	mat[1:3,1:3]=COS*id+SIN*Ux+(1.0-COS)*UU
	return mat
end

rAxis (generic function with 1 method)

In [13]:
@btime r(1,1,1)

  722.414 ns (9 allocations: 992 bytes)


4×4 MMatrix{4, 4, Float64, 16} with indices SOneTo(4)×SOneTo(4):
 1.0      0.17353  2.14758  0.0
 2.14758  1.0      0.17353  0.0
 0.17353  2.14758  1.98703  0.0
 0.0      0.0      0.0      1.0

In [14]:
@btime r(0)

  18.355 ns (1 allocation: 80 bytes)


3×3 MMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 1.0  -0.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0

In [15]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 10000 samples with 134 evaluations.
 Range (min … max):  699.254 ns … 38.169 μs  ┊ GC (min … max): 0.00% … 92.27%
 Time  (median):     804.478 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   944.009 ns ±  1.170 μs  ┊ GC (mean ± σ):  6.18% ±  5.07%

  ▆▇██▆▆▄▂▂       ▁▁▁▁                            ▂            ▂
  █████████▇████▇██████▇██▇▇█▇▅▆▅▅▄▁▄▅▅▃▄▄▄▃▄▅▁▅▄▄█▇▇▅▅▇▇▆▇▅▅▄ █
  699 ns        Histogram: log(frequency) by time      2.69 μs <

 Memory estimate: 992 bytes, allocs estimate: 9.

In [16]:
@benchmark r(0)

BenchmarkTools.Trial: 10000 samples with 997 evaluations.
 Range (min … max):  19.057 ns …   4.172 μs  ┊ GC (min … max):  0.00% … 98.01%
 Time  (median):     20.361 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   27.820 ns ± 102.597 ns  ┊ GC (mean ± σ):  14.67% ±  4.03%

  ▇█▆▃▂▂▂▁ ▂▃▂▁     ▁                                          ▂
  ███████████████████▇▆▆▆▆▄▆▅▆▄▅▅▅▅▃▃▄▃▁▁▃▁▃▁▅▄▇▆████▅█▇▆▇▇▅█▆ █
  19.1 ns       Histogram: log(frequency) by time      74.5 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# using inline in the r funcion we can cut the 2D mean time by 4ns and 100ns for 3D

In [17]:
@inline function r(args...)
   n = length(args)

   if n == 1 # rotation in 2D
	   mat = r2D(args)
   end

    if n == 3 # rotation in 3D
       mat = r3D(args)
	end
	return mat
end

r (generic function with 1 method)

In [18]:
@benchmark r(1,1,1)

BenchmarkTools.Trial: 10000 samples with 179 evaluations.
 Range (min … max):  560.894 ns …  18.162 μs  ┊ GC (min … max): 0.00% … 95.71%
 Time  (median):     639.665 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   710.805 ns ± 803.527 ns  ┊ GC (mean ± σ):  6.51% ±  5.62%

    ▂█ ▇ ▆                                                       
  ▄▆██▆███▆█▅▇▅▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  561 ns           Histogram: frequency by time         1.25 μs <

 Memory estimate: 944 bytes, allocs estimate: 7.

In [19]:
@benchmark r(0)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  13.413 ns …   3.193 μs  ┊ GC (min … max):  0.00% … 97.72%
 Time  (median):     14.715 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   21.972 ns ± 103.205 ns  ┊ GC (mean ± σ):  18.92% ±  4.04%

  ▆█▆▄▃▂▁▁ ▁▂▂▁                                    ▁ ▁         ▂
  ███████████████▇█▇▇▇▆▆▆▆▅▄▆▄▁▄▃▁▄▃▃▄▁▃▁▁▁▁▁▁▄▅▅▇▆█▅█▇▆█▆█▇▅▇ █
  13.4 ns       Histogram: log(frequency) by time      66.7 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.